### Synthetic Data Backtest

Most of the functions below can be found under:

* Tool/backtest 
* Sample_data/make_data

A quick slideshow: [Seven Sins of Backtesting](https://newyork.qwafafew.org/wp-content/uploads/sites/4/2015/10/Luo_20150128.pdf)

The below is a code snippet for trading rules optimization. As for how to use it, I leave it to the reader.

The key for this exercise was to use real-HFT data to deduce a certain outcome.. which I don't have.

Hence, I will just replace this exercise with multiprocessing methods (Parallelization!).

If you are keen on generating synthetic data for your research, copy the code snippets [Generate synthetic raw data](https://gist.github.com/boyboi86/5e00faf48f60abfdbe838fbdee269471) in my gist.

Contact: boyboi86@gmail.com

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import research as rs
from random import gauss
from itertools import product

%matplotlib inline

Num of CPU core:  4
Machine info:  Windows-10-10.0.18362-SP0
Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy 1.18.5
Pandas 1.0.4


C:\Users\Wei_X\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<Figure size 1500x800 with 1 Axes>

In [2]:
dict0 = {'a':['1', '2'], 'b':['+', '*'], 'c':['!','A']}
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print({'a':a, 'b':b, 'c':c})

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

The above is scalar operation. Notice loop after loop after loop.

The below is vectorization operation, which is what we will do for Monte-Carlos.

The result is actually the same.

In [3]:
from itertools import product
jobs = (dict(zip(dict0, i )) for i in product(*dict0.values()))
for i in jobs: print(i)

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

Given the parameters below. It should look like the below.

In [4]:
forecast = [10,5,0,-5,-10]
half_life = [5,10,25,50,100]
loss_range = profit_range = np.linspace(.5,10,20)
sigma = [1]

coeff = dict({'forecast': forecast,
              'half_life': half_life,
              'sigma':sigma,
              'profit_range': profit_range,
              'loss_range':loss_range})
jobs = list(dict(zip(coeff, idx)) for idx in product(*coeff.values()))

print(len(jobs))
out = pd.DataFrame(jobs)
idx = 0
out

10000


,forecast,half_life,sigma,profit_range,loss_range
0,10,5,1,0.5,0.5
1,10,5,1,0.5,1.0
2,10,5,1,0.5,1.5
3,10,5,1,0.5,2.0
4,10,5,1,0.5,2.5
...,...,...,...,...,...
9995,-10,100,1,10.0,8.0
9996,-10,100,1,10.0,8.5
9997,-10,100,1,10.0,9.0
9998,-10,100,1,10.0,9.5


**Note**

The below is the end product for a parallized Monte-Carlos Simulation, for optimizied Trading Rule Algorithm.

Primarily we will use it to create scenarios for our models to be backtest with.

In [5]:
def _opt_tr(coeff: dict = None,
            n_run: float = 100,
            max_period: int = 100,
            seed: int = 0):
    
    phi, output1 = 2 ** (-1/ coeff['half_life']), []
    for iter_ in range(int(n_run)):
        p, hold_period, count = seed, 0, 0
        while True:
            p = (1 - phi) * coeff['forecast'] + phi * p + coeff['sigma'] * gauss(0,1)
            cp = p - seed
            hold_period += 1
            if coeff['profit_range'] < cp or -coeff['loss_range'] > cp or hold_period > max_period:
                output1.append(cp)
                break
    mean, std = np.mean(output1), np.std(output1)
    return {'profit': coeff['profit_range'], 'loss': coeff['loss_range'], 'mean': mean, 'std': std, 'sharpe': mean/std}

def opt_tr(profit_range: list = np.linspace(.5, 10, 20),
          loss_range: list = np.linspace(.5, 10, 20),
          sigma: list = [1],
          n_run: float = 100,
          max_period: int = 100,
          forecast: list = [10,5,0,-5,-10], 
          half_life: list = [5,10,25,50,100],
          num_threads: int = 1):
    
    if profit_range is None:
        profit_range = np.linspace(0,10,21)
        
    if loss_range is None:
        loss_range = np.linspace(0,10,21)
        
    _coeff_list = []
    _coeff = dict({'forecast': forecast,
                   'half_life': half_life,
                   'sigma':sigma,
                   'profit_range': profit_range,
                   'loss_range':loss_range})
    
    _coeff = list(dict(zip(_coeff, idx )) for idx in product(*_coeff.values()))
    out = []
    for coeff in _coeff:
        jobs = [{'func': _opt_tr,
                   'coeff': coeff,
                   'n_run': n_run, 
                   'max_period': max_period}]
        
        if num_threads == 1:
            out_ = rs.process_jobs_(jobs)
            out.append(out_)
        else:
            out_ = rs.process_jobs(jobs, num_threads = num_threads)
            out.append(out_)

    return out

**Note**

Before parallization. Total runs for estimated 1,000,000 loops took only less than 1 min

* 2020-06-14 17:02:17.058014 (Beginning!)

* 2020-06-14 17:03:06.231600 (End!)

**Note**

Try to change the above to run parallelization.

Please do not use jupyter notbook, it won't reflect anything.

If it is already fast, like the above. Chances are the algorithm will become incredibly slow.

**Hint**

Try using "iterrows()"

In [6]:
df = opt_tr()

Expand call {'profit': 0.5, 'loss': 0.5, 'mean': 1.4493359235879852, 'std': 1.048823247531226, 'sharpe': 1.3818686103683404}
Expand call {'profit': 0.5, 'loss': 1.0, 'mean': 1.522315933716676, 'std': 0.7848377096357615, 'sharpe': 1.9396569698761974}
Expand call {'profit': 0.5, 'loss': 1.5, 'mean': 1.6284868031281474, 'std': 0.871498799880418, 'sharpe': 1.8686047569447013}
Expand call {'profit': 0.5, 'loss': 2.0, 'mean': 1.7033043808962751, 'std': 0.7161708144649716, 'sharpe': 2.3783493357918526}
Expand call {'profit': 0.5, 'loss': 2.5, 'mean': 1.4619547309102217, 'std': 0.6475207991682549, 'sharpe': 2.257772619486684}
Expand call {'profit': 0.5, 'loss': 3.0, 'mean': 1.5734586079493769, 'std': 0.7007301380637215, 'sharpe': 2.2454558787741092}
Expand call {'profit': 0.5, 'loss': 3.5, 'mean': 1.8353711477461574, 'std': 0.874203182194788, 'sharpe': 2.0994789142018977}
Expand call {'profit': 0.5, 'loss': 4.0, 'mean': 1.5002853538176362, 'std': 0.7086448478218627, 'sharpe': 2.117118833826298

2020-06-14 21:12:09.309669 done after 0.00 mins
2020-06-14 21:12:09.310666 done after 0.00 mins
2020-06-14 21:12:09.313658 done after 0.00 mins
2020-06-14 21:12:09.315653 done after 0.00 mins
2020-06-14 21:12:09.319648 done after 0.00 mins
2020-06-14 21:12:09.322635 done after 0.00 mins
2020-06-14 21:12:09.336599 done after 0.00 mins
2020-06-14 21:12:09.337595 done after 0.00 mins
2020-06-14 21:12:09.339595 done after 0.00 mins
2020-06-14 21:12:09.341584 done after 0.00 mins
2020-06-14 21:12:09.342581 done after 0.00 mins
2020-06-14 21:12:09.343578 done after 0.00 mins
2020-06-14 21:12:09.344576 done after 0.00 mins
2020-06-14 21:12:09.345573 done after 0.00 mins
2020-06-14 21:12:09.346570 done after 0.00 mins
2020-06-14 21:12:09.348566 done after 0.00 mins
2020-06-14 21:12:09.349563 done after 0.00 mins
2020-06-14 21:12:09.352555 done after 0.00 mins
2020-06-14 21:12:09.356550 done after 0.00 mins
2020-06-14 21:12:09.358539 done after 0.00 mins
2020-06-14 21:12:09.359536 done after 0.

Expand call {'profit': 3.0, 'loss': 4.5, 'mean': 3.883169441231285, 'std': 0.7093071904741601, 'sharpe': 5.474594778371626}
Expand call {'profit': 3.0, 'loss': 5.0, 'mean': 3.8268511538625334, 'std': 0.6537258139986089, 'sharpe': 5.853908583562033}
Expand call {'profit': 3.0, 'loss': 5.5, 'mean': 3.776890404441767, 'std': 0.6178924599059349, 'sharpe': 6.112536807807532}
Expand call {'profit': 3.0, 'loss': 6.0, 'mean': 3.9137574310123715, 'std': 0.62506705421622, 'sharpe': 6.261340130811861}
Expand call {'profit': 3.0, 'loss': 6.5, 'mean': 3.8738161709629675, 'std': 0.6634267433994726, 'sharpe': 5.839101618233087}
Expand call {'profit': 3.0, 'loss': 7.0, 'mean': 3.873103373194704, 'std': 0.7158919004271976, 'sharpe': 5.41017906597838}
Expand call {'profit': 3.0, 'loss': 7.5, 'mean': 3.956084292275561, 'std': 0.7202674469310854, 'sharpe': 5.492521297653587}
Expand call {'profit': 3.0, 'loss': 8.0, 'mean': 3.931038754845686, 'std': 0.6065784219093666, 'sharpe': 6.480676880116668}
Expand c

2020-06-14 21:12:09.498328 done after 0.00 mins
2020-06-14 21:12:09.500320 done after 0.00 mins
2020-06-14 21:12:09.503312 done after 0.00 mins
2020-06-14 21:12:09.505308 done after 0.00 mins
2020-06-14 21:12:09.508299 done after 0.00 mins
2020-06-14 21:12:09.510294 done after 0.00 mins
2020-06-14 21:12:09.512288 done after 0.00 mins
2020-06-14 21:12:09.514283 done after 0.00 mins
2020-06-14 21:12:09.516277 done after 0.00 mins
2020-06-14 21:12:09.519270 done after 0.00 mins
2020-06-14 21:12:09.528250 done after 0.00 mins
2020-06-14 21:12:09.531239 done after 0.00 mins
2020-06-14 21:12:09.533233 done after 0.00 mins
2020-06-14 21:12:09.536223 done after 0.00 mins
2020-06-14 21:12:09.538218 done after 0.00 mins
2020-06-14 21:12:09.541215 done after 0.00 mins
2020-06-14 21:12:09.543205 done after 0.00 mins
2020-06-14 21:12:09.546197 done after 0.00 mins
2020-06-14 21:12:09.548192 done after 0.00 mins
2020-06-14 21:12:09.551185 done after 0.00 mins
2020-06-14 21:12:09.554177 done after 0.

 {'profit': 4.5, 'loss': 3.0, 'mean': 5.368145442664643, 'std': 0.5959796533653106, 'sharpe': 9.007262936498597}
Expand call {'profit': 4.5, 'loss': 3.5, 'mean': 5.269913497314059, 'std': 0.6126155115059141, 'sharpe': 8.60231809077068}
Expand call {'profit': 4.5, 'loss': 4.0, 'mean': 5.2881469585714065, 'std': 0.6193833610067434, 'sharpe': 8.537760765765602}
Expand call {'profit': 4.5, 'loss': 4.5, 'mean': 5.237374684198681, 'std': 0.6970978188968467, 'sharpe': 7.51311299824004}
Expand call {'profit': 4.5, 'loss': 5.0, 'mean': 5.270255955075134, 'std': 0.5977020320947491, 'sharpe': 8.817530595645826}
Expand call {'profit': 4.5, 'loss': 5.5, 'mean': 5.211061743607128, 'std': 0.540275632757197, 'sharpe': 9.64519113514973}
Expand call {'profit': 4.5, 'loss': 6.0, 'mean': 5.283613775104083, 'std': 0.6753775264307579, 'sharpe': 7.823200459492604}
Expand call {'profit': 4.5, 'loss': 6.5, 'mean': 5.32516415603729, 'std': 0.6006610132486437, 'sharpe': 8.865506564570284}
Expand call {'profit': 

...

2020-06-14 21:13:06.562170 done after 0.00 mins
2020-06-14 21:13:06.570146 done after 0.00 mins
2020-06-14 21:13:06.577129 done after 0.00 mins
2020-06-14 21:13:06.587102 done after 0.00 mins
2020-06-14 21:13:06.597074 done after 0.00 mins
2020-06-14 21:13:06.606603 done after 0.00 mins
2020-06-14 21:13:06.613072 done after 0.00 mins
2020-06-14 21:13:06.621015 done after 0.00 mins
2020-06-14 21:13:06.630988 done after 0.00 mins
2020-06-14 21:13:06.639078 done after 0.00 mins
2020-06-14 21:13:06.639078 done after 0.00 mins
2020-06-14 21:13:06.654702 done after 0.00 mins
2020-06-14 21:13:06.654702 done after 0.00 mins
2020-06-14 21:13:06.670359 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.685943 done after 0.00 mins
2020-06-14 21:13:06.701564 done after 0.

Expand call {'profit': 9.5, 'loss': 5.5, 'mean': -3.4326119465357783, 'std': 5.410878030608701, 'sharpe': -0.6343909301074421}
Expand call {'profit': 9.5, 'loss': 6.0, 'mean': -3.4379486803350967, 'std': 5.59871189716253, 'sharpe': -0.6140606524292624}
Expand call {'profit': 9.5, 'loss': 6.5, 'mean': -3.642826029616693, 'std': 5.871835952720333, 'sharpe': -0.6203896122011083}
Expand call {'profit': 9.5, 'loss': 7.0, 'mean': -3.6482506760146465, 'std': 5.887464106160716, 'sharpe': -0.6196641899178751}
Expand call {'profit': 9.5, 'loss': 7.5, 'mean': -4.441751332274993, 'std': 6.202658396695559, 'sharpe': -0.7161044584756301}
Expand call {'profit': 9.5, 'loss': 8.0, 'mean': -4.332078447019941, 'std': 6.064848700185802, 'sharpe': -0.7142929133396557}
Expand call {'profit': 9.5, 'loss': 8.5, 'mean': -4.571935549025232, 'std': 6.005616739193746, 'sharpe': -0.7612766094759178}
Expand call {'profit': 9.5, 'loss': 9.0, 'mean': -3.453291316837362, 'std': 7.429866026114124, 'sharpe': -0.46478513

2020-06-14 21:13:06.763316 done after 0.00 mins
2020-06-14 21:13:06.778273 done after 0.00 mins
2020-06-14 21:13:06.789246 done after 0.00 mins
2020-06-14 21:13:06.799217 done after 0.00 mins
2020-06-14 21:13:06.806307 done after 0.00 mins
2020-06-14 21:13:06.822092 done after 0.00 mins
2020-06-14 21:13:06.838046 done after 0.00 mins
2020-06-14 21:13:06.841099 done after 0.00 mins
2020-06-14 21:13:06.843068 done after 0.00 mins
2020-06-14 21:13:06.846061 done after 0.00 mins
2020-06-14 21:13:06.849052 done after 0.00 mins
2020-06-14 21:13:06.853047 done after 0.00 mins
2020-06-14 21:13:06.859987 done after 0.00 mins
2020-06-14 21:13:06.863976 done after 0.00 mins
2020-06-14 21:13:06.869961 done after 0.00 mins
2020-06-14 21:13:06.873010 done after 0.00 mins
2020-06-14 21:13:06.873010 done after 0.00 mins
2020-06-14 21:13:06.888670 done after 0.00 mins
2020-06-14 21:13:06.888670 done after 0.00 mins
2020-06-14 21:13:06.904252 done after 0.00 mins
2020-06-14 21:13:06.904252 done after 0.

Expand call {'profit': 10.0, 'loss': 8.5, 'mean': -3.327765931712461, 'std': 7.054322562318431, 'sharpe': -0.4717343022401824}
Expand call {'profit': 10.0, 'loss': 9.0, 'mean': -3.4492145382144024, 'std': 7.3404872817032, 'sharpe': -0.469889042218201}
Expand call {'profit': 10.0, 'loss': 9.5, 'mean': -5.029749962400315, 'std': 6.745830931424596, 'sharpe': -0.7456086601533198}
Expand call {'profit': 10.0, 'loss': 10.0, 'mean': -4.377969111530769, 'std': 7.247222469836397, 'sharpe': -0.6040892396710984}


2020-06-14 21:13:06.997302 done after 0.00 mins


In [7]:
df

[[{'profit': 0.5,
   'loss': 0.5,
   'mean': 1.4493359235879852,
   'std': 1.048823247531226,
   'sharpe': 1.3818686103683404}],
 [{'profit': 0.5,
   'loss': 1.0,
   'mean': 1.522315933716676,
   'std': 0.7848377096357615,
   'sharpe': 1.9396569698761974}],
 [{'profit': 0.5,
   'loss': 1.5,
   'mean': 1.6284868031281474,
   'std': 0.871498799880418,
   'sharpe': 1.8686047569447013}],
 [{'profit': 0.5,
   'loss': 2.0,
   'mean': 1.7033043808962751,
   'std': 0.7161708144649716,
   'sharpe': 2.3783493357918526}],
 [{'profit': 0.5,
   'loss': 2.5,
   'mean': 1.4619547309102217,
   'std': 0.6475207991682549,
   'sharpe': 2.257772619486684}],
 [{'profit': 0.5,
   'loss': 3.0,
   'mean': 1.5734586079493769,
   'std': 0.7007301380637215,
   'sharpe': 2.2454558787741092}],
 [{'profit': 0.5,
   'loss': 3.5,
   'mean': 1.8353711477461574,
   'std': 0.874203182194788,
   'sharpe': 2.0994789142018977}],
 [{'profit': 0.5,
   'loss': 4.0,
   'mean': 1.5002853538176362,
   'std': 0.7086448478218627,


In [8]:
len(df)

10000